In [132]:
import json
import random
from pathlib import Path
import collections

import pandas as pd
import numpy as np

%load_ext autoreload
%autoreload 2
Path.ls = lambda x: list(x.iterdir())
# from urllib.parse import quote_plus

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [133]:
# column names in JHU format
REGION = 'Province/State'

In [134]:
from datetime import date

today = date.today()

# dd/mm/YY
current_date = today.strftime("%Y%m%d")
print("d1 =", current_date)

d1 = 20200414


In [135]:
import urllib.request, json
from urllib.error import HTTPError

# def get_raw_data(raw_web_url="https://api.covid19india.org/raw_data.json"):
#     with urllib.request.urlopen(raw_web_url) as url:
#         data_dict = json.loads(url.read().decode())
#         return data_dict["raw_data"]


def get_stats_history(stats_history_url="https://api.rootnet.in/covid19-in/stats/history"):
    try:
        with urllib.request.urlopen(stats_history_url) as url:
            data_dict = json.loads(url.read().decode())
            return data_dict
    except HTTPError as e:
        print(f"Using local backup of {stats_history_url}")
        with open("history.json") as f:
            return json.loads(f.read())

def get_stats_history_district(stats_history_url="https://api.covid19india.org/raw_data.json"):
    try:
        with urllib.request.urlopen(stats_history_url) as url:
            data_dict = json.loads(url.read().decode())
            return data_dict
    except HTTPError as e:
        print(f"Using local backup of {stats_history_url}")
        with open("history.json") as f:
            return json.loads(f.read())
        
# def get_state_data(
#     case_count_url="https://api.covid19india.org/state_district_wise.json",
# ):
#     with urllib.request.urlopen(case_count_url) as url:
#         data_dict = json.loads(url.read().decode())
#         return data_dict


# def get_case_count(data, state="Karnataka"):
#     df = pd.DataFrame(data[state]["districtData"])
#     df = df.transpose()
#     df.reset_index(inplace=True)
#     df.rename(columns={"confirmed": "CaseCount", "index": "District"}, inplace=True)
#     df.drop(columns=["lastupdatedtime"], inplace=True)
#     return df

In [136]:
from typing import List, Dict
#stats:List = get_stats_history()["data"]
stats:List = get_stats_history_district()["raw_data"]

In [137]:
district_df = pd.DataFrame([(i['detecteddistrict'],i['dateannounced']) for i in stats])
district_df.columns = [REGION,'date']

In [138]:
import pandas as pd
from datetime import datetime
date_list = pd.date_range(start="2020-01-22",end=datetime.today()).strftime("%d/%m/%Y")

In [140]:
district_df_pivot = pd.pivot_table(district_df,values=REGION,index=[REGION],columns=['date'],aggfunc=np.count_nonzero,fill_value=0)
district_df_pivot_dated = district_df_pivot.reindex(date_list, axis=1).fillna(0)
district_df_final = district_df_pivot_dated.cumsum(axis=1)

## Transform to the JHU data format
See this [Github link to data](https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv) for reference

In [141]:
district_df_final.insert(0, "Country/Region", "India")
district_df_final.insert(1, "Lat", 20)
district_df_final.insert(2, "Long", 70)

In [142]:
district_df_final.head()

,Country/Region,Lat,Long,22/01/2020,23/01/2020,24/01/2020,25/01/2020,26/01/2020,27/01/2020,28/01/2020,...,05/04/2020,06/04/2020,07/04/2020,08/04/2020,09/04/2020,10/04/2020,11/04/2020,12/04/2020,13/04/2020,14/04/2020
Province/State,,,,,,,,,,,,,,,,,,,,,
,India,20,70,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,536.0,559.0,621.0,723.0,778.0,974.0,1166.0,1287.0,1722.0,1882.0
Adilabad,India,20,70,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,2.0,4.0,4.0,4.0,20.0,20.0,20.0,20.0,20.0
Agra,India,20,70,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,94.0,98.0,124.0,128.0,166.0,176.0,184.0,208.0,278.0,278.0
Ahmadabad,India,20,70,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,106.0,128.0,166.0,166.0,284.0,394.0,486.0,564.0,640.0,702.0
Ahmadnagar,India,20,70,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,42.0,46.0,50.0,50.0,50.0,50.0,52.0,52.0,54.0,54.0


In [143]:
fname = f'../data/time_series_covid19_confirmed_India_district_{current_date}.csv'
district_df_final.to_csv(fname, header=True)

In [144]:
!cat $fname

Province/State,Country/Region,Lat,Long,22/01/2020,23/01/2020,24/01/2020,25/01/2020,26/01/2020,27/01/2020,28/01/2020,29/01/2020,30/01/2020,31/01/2020,01/02/2020,02/02/2020,03/02/2020,04/02/2020,05/02/2020,06/02/2020,07/02/2020,08/02/2020,09/02/2020,10/02/2020,11/02/2020,12/02/2020,13/02/2020,14/02/2020,15/02/2020,16/02/2020,17/02/2020,18/02/2020,19/02/2020,20/02/2020,21/02/2020,22/02/2020,23/02/2020,24/02/2020,25/02/2020,26/02/2020,27/02/2020,28/02/2020,29/02/2020,01/03/2020,02/03/2020,03/03/2020,04/03/2020,05/03/2020,06/03/2020,07/03/2020,08/03/2020,09/03/2020,10/03/2020,11/03/2020,12/03/2020,13/03/2020,14/03/2020,15/03/2020,16/03/2020,17/03/2020,18/03/2020,19/03/2020,20/03/2020,21/03/2020,22/03/2020,23/03/2020,24/03/2020,25/03/2020,26/03/2020,27/03/2020,28/03/2020,29/03/2020,30/03/2020,31/03/2020,01/04/2020,02/04/2020,03/04/2020,04/04/2020,05/04/2020,06/04/2020,07/04/2020,08/04/2020,09/04/2020,10/04/2020,11/04/2020,12/04/2020,13/04/2020,14/04/2020
,India,20,70,0.0,0.0,0.0,0.0,0.0,0.0,